# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.


In [3]:
"""Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    coef :: Array{T,1}     #Arreglo de coeficientes
    order :: Int64         # Orden del polinimio
end


In [4]:
#Constructores para el tipo Taylor
Taylor{T<:Number}(v::Array{T,1}) = Taylor(v, length(v))   #Recibe un arreglo de coeficientes
Taylor(N::Int64) = Taylor(zeros(N), N) #Recibe solo el orden y llena con ceros.

Taylor{T<:Number}

Como necesitamos una función que permita _igualar_ el grado de dos objetos tipo Taylor. Propones la función __upgrade__.

In [5]:
function upgrade(a::Taylor, b::Taylor)
    order_a = a.order
    order_b = b.order
    dif = order_a - order_b    #Calcula la diferencia entre los ordenes de dos Taylor
    
    if dif >= 0     
        for i in 1:abs(dif)
            push!(b.coef, 0)  #Si el orden de b es menor, agrega tantos ceros como hagan falta para igualar a a.
        end
        b.order = a.order
    else
        for i in 1:abs(dif)
            push!(a.coef, 0)  # Lo mismo si a es de orden menor.
        end
        a.order = b.order
    end
    return a,b
        
    
    
end

upgrade (generic function with 1 method)

Ahora definimos las operaciones básicas

In [6]:
import Base: +, -, *, /, ==

In [7]:
# Para la suma y la resta usamos metaprogramming

for fn = (:+, :-)
    ex = quote
        function ($fn)(a::Taylor, b::Taylor)
            c,d = upgrade(a,b)     # Hace un upgrade, asi no es tragico si los Taylor tienen orden diferente.
            s = Taylor(c.order)
            for i in 1:s.order
                s.coef[i] = ($fn)(c.coef[i], d.coef[i])
            end
            return s
        end
    end
        @eval $ex
end

In [8]:
function /(a::Taylor, b::Taylor)
    f, g = upgrade(a,b)
    k = f.order
    l = g.order
    r = 0
    for j in 1:l
        if g.coef[j] != 0 
            r = j
            break
        end
    end
    @assert r != 0 "Requerimos que g sea distinto de 0"
    h = Taylor(k)
    #@show g
    h.coef[1] = f.coef[r] / g.coef[r]
    
    for j in (r + 1):k
        #@show j
        f_j = f.coef[j]
        suma = 0
        for i in 1:(j - r)
            #@show i
            h_i = h.coef[i]
            g_i = g.coef[j - i + 1]
            suma += h_i * g_i
        end
        h.coef[j + 1 - r] = (f_j - suma)/g.coef[r]
    end
    return h
end

/ (generic function with 53 methods)

In [9]:
function *(a::Taylor, b::Taylor)
    f,g = upgrade(a,b)
    h = Taylor(f.order)
    
    for k in 1:f.order
        s = 0
        for i in 1:k
            
            h.coef[k] += f.coef[i]*g.coef[(k+1) - i]
            #println(k, i, "  ", s)
            
        
        end
        
    end
    return h
            
end



* (generic function with 143 methods)

In [10]:
function *{T<:Number}(f::Taylor, α::T)
    h = Taylor(zeros(typeof(promote(α, f.coef[1])[1]), f.order))
    
    for k in 1:f.order
        h.coef[k] = α * f.coef[k]        
    end
    return h
            
end
*{T<:Number}(α::T, f::Taylor) = *(f, α)

* (generic function with 145 methods)

In [11]:
function ==(a::Taylor, b::Taylor)
    f,g = upgrade(a,b)
    r = true
    for k in 1:f.order
        if g.coef[k] != f.coef[k]
            r = false
            break
        end
    end
    return r
end
            

== (generic function with 112 methods)

In [12]:
using Base.Test
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
# Antes de los random, usemos algo que sepamos va a funcionar, la funcion f(x) = x^2 - 1 y g(x) = x - 1

T1 = Taylor([-1, 0, 1, 0]) # x^2 - 1
T2 = Taylor([-1, 1, 0 ,0]) # -1 + x



@test (T1+T2) == Taylor([-2, 1, 1, 0])
@test (T1-T2) == Taylor([0, -1, 1, 0])
@test (T2*T1) == Taylor([1, -1, -1, 1])
@test (T1/T2) == Taylor([1, 1, 0, 0])
@test 2*T1    == Taylor([-2.0,0.0,2.0,0.0])
@test T1*2    == Taylor([-2.0,0.0,2.0,0.0])
@test im*T1   == Taylor([0 - 1im,0 + 0im,0 + 1im,0 + 0im])
@test im*T1   == T1*im

# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

**NOTA** Los ejercicios de este notebook constituyen el contenido de la Tarea6.

## Para $E(x)$

Tomando $g(x) = \sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}$, $E(x) = exp(g(x)) = \sum_{k=1}^{\infty}E_k(x-x_0)^{k-1}$:

\begin{align}
    \frac{{\rm d}E}{{\rm d}x} &= E(x) g'(x)\\
    \Leftrightarrow \sum_{k = 1}^{\infty}kE_{k+1}(x-x_0)^{k-1} &= \left(\sum_{k = 1}^{\infty}E_k(x-x_0)^{k-1} \right)\left(\sum_{j = 1}^{\infty}kg_{k+1}(x-x_0)^{k-1} \right) \\
    \Leftrightarrow \sum_{k = 1}^{\infty}kE_{k+1}(x-x_0)^{k-1} &= \sum_{k = 1}^{\infty}\left(\sum_{j=1}^kE_j(k-j)g_{k-j+1} \right)(x-x_0)^{k-1} \\
    \Leftrightarrow E_{k+1} &= \frac{1}{k}\sum_{j=1}^kE_j(k-j)g_{k-j+1} \\
    \Leftrightarrow E_{k} &= \frac{1}{k-1}\sum_{j=1}^{k-1}E_j(k-j)g_{k-j+1}
\end{align}

## Para $P_a(x)$ 

Hagamos las cuentas, definiendo a los polinomios $g(x) = \sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}$, $P_a(x) = (g(x))^a = \sum_{k=1}^{\infty}P_{a_{k}}(x-x_0)^{k-1}$ (para considerar que los arreglos en `Julia` se contabilizan desde 1) tenemos que:

\begin{align}
    \frac{{\rm d}P_a}{{\rm d}x} &= a P_{a-1}(x) g'(x)\\
    \Leftrightarrow g(x)\frac{{\rm d}P_a}{{\rm d}x}&= a P_a(x) g'(x)\\
\end{align}

Desarrollando el lado izquierdo de la ecuación:
\begin{align}
    g(x)\frac{{\rm d}P_a}{{\rm d}x}&= \left(\sum_{j=2}^{\infty}(j-1)P_{a_{j}}(x-x_0)^{j-2}\right) \left(\sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}\right) \\
    &= \left(\sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}\right)\left(\sum_{j=1}^{\infty}jP_{a_{j+1}}(x-x_0)^{j-1}\right)\\
    &= \sum_{k=1}^{\infty}\left(\sum_{j=1}^{k}g_j(k-j)P_{a_{k-j+1}}\right)(x-x_0)^{k-1}
\end{align}

Por otra parte, del lado derecho de la ecuación:
\begin{align}
    a P_a(x) g'(x)&= a\left(\sum_{k=1}^{\infty}P_k(x-x_0)^{k-1}\right)\left(\sum_{j=2}^{\infty}(j-1)g_j(x-x_0)^{j-2}\right) \\
    &= a\left(\sum_{k=1}^{\infty}P_{a_k}(x-x_0)^{k-1}\right)\left(\sum_{j=1}^{\infty}jg_{j+1}(x-x_0)^{j-1}\right)\\
    &= a\sum_{k=1}^{\infty}\left(\sum_{j=1}^{k}P_{a_j}(k-j)g_{k-j+1}\right)(x-x_0)^{k-1}
\end{align}

Por igualdad de polinomios tendremos que:
\begin{align}
    \sum_{j=1}^{k}g_j(k-j)P_{a_{k-j+1}} &= a\sum_{j=1}^{k}P_{a_j}(k-j)g_{k-j+1}\\
    \Leftrightarrow (k-1)g_1P_{a_k} &= a\sum_{j=1}^{k}P_{a_j}(k-j)g_{k-j+1} - \sum_{j=2}^{k}g_j(k-j)P_{a_{k-j+1}}\\
    \Leftrightarrow P_{a_k} &= \frac{1}{(k-1)g_1}\left( a\sum_{j=1}^{k}P_{a_j}(k-j)g_{k-j+1} - \sum_{j=3}^{k}g_j(k-j)P_{a_{k-j+1}}\right) \ \ \forall k > 1\\
\end{align}

Para $k = 1$ tendremos que $P_{a_1} = (g_1)^a$ por lo tanto:

\begin{equation}
    P_{a_k} = \left\{ \begin{array}{rl}
    g_1^a & \text{si } k = 1\\
    \frac{1}{(k-1)g_1}\left( a\sum_{j=1}^{k-1}P_{a_j}(k-j)g_{k-j+1} - \sum_{j=3}^{k-1}g_j(k-j)P_{a_{k-j+1}}\right) & \text{si } k > 1\\
    \end{array} \right.
\end{equation}

In [13]:
#Para la potencia
import Base.^

In [14]:
function ^(a::Taylor, b::Int)
    h = Taylor((a.order - 1)* b + 1) #Creamos el nuevo Taylor 
    P, g = upgrade(h, a) #Hacemos que a y h tengan el mismo orden
    r = 0
    for i in 1:a.order #Verificamos que el Taylor proporcionado sea distinto de cero
        if g.coef[i] != 0
            r = i
            break
        end
    end
    @assert r != 0 "Necesitamos un polinomio distinto de cero"
    #@show a
    P.coef[1] = g.coef[1]^b
    
    
    for k in 2:h.order
        suma_1 = 0
        suma_2 = 0

        for j in 1:(k-1)
            g_j = g.coef[k + 1 - j]
            P_j = P.coef[j]
            suma_1 += (k - j)*g_j*P_j
            
        end

        for j in 2:(k - 1)
            g_j = g.coef[j]
            P_q = P.coef[k + 1 - j]
            suma_2 += (k - j)*g_j*P_q
        end
        P.coef[k] = (b*suma_1 - suma_2)/((k - 1)*g.coef[1])
    end
    
    return P
end            

^ (generic function with 52 methods)

## Para $L(x)$

Hagamos las cuentas para obtener los coeficientes $L_k$. Partamos de $g(x) = \sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}$, $L(x) = \log(g(x)) = \sum_{k=1}^{\infty}L_k(x-x_0)^{k-1}$

Observemos que:

\begin{align}
    \frac{{\rm d}L}{{\rm d}x} &= \frac{g'(x)}{g(x)}\\
    \Leftrightarrow g(x)\frac{{\rm d}L}{{\rm d}x}&= g'(x)\\
\end{align}

Sustituyendo las ecuaciones con su respectiva representación en series tendremos que:
\begin{align}
    \sum_{k=2}^{\infty}(k-1)g_j(x-x_0)^{k-2} &= \left(\sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}\right)\left(\sum_{j=2}^{\infty}(j-1)L_j(x-x_0)^{j-2}\right) \\
    \Leftrightarrow \sum_{k=1}^{\infty}kg_{k+1}(x-x_0)^{k-1} &= \left(\sum_{k=1}^{\infty}g_k(x-x_0)^{k-1}\right)\left(\sum_{j=1}^{\infty}jL_{j+1}(x-x_0)^{j-1}\right) \\
    \Leftrightarrow \sum_{k=1}^{\infty}kg_{k+1}(x-x_0)^{j-1} &= \sum_{k=1}^{\infty}\left(\sum_{j=1}^{k}g_j(k-j)L_{k-j + 1}\right)(x-x_0)^{k-1}\\
    \Leftrightarrow kg_{k+1} &= \sum_{j=1}^{k}g_j(k-j)L_{k-j+1}\\
    \Leftrightarrow g_1(k-1)L_k &= kg_{k+1} - \sum_{j=2}^{k}g_j(k-j)L_{k-j+1}\\
    \Leftrightarrow L_k &= \frac{1}{g_1(k-1)}\left(kg_{k+1} - \sum_{j=2}^{k}g_j(k-j)L_{k-j+1}\right) \ \ \forall k >1
\end{align}

In [15]:
#El Logaritmo
import Base.log

In [16]:
function log(g::Taylor)
    L = Taylor(g.order)
    r = 0
    
    for i in 1:g.order
        if g.coef[i] != 0
            r = i
            
            break
        end
    end
    @assert r != 0 "Necesitamos un polinomio distinto de cero"
    #@show r
    g_r = g.coef[r]
    L.coef[r] = log(g_r)
    
    for k in (r + 1):g.order
        
        k != g.order ? g_k = g.coef[k + 1] : g_k = 0
        suma = 0
        for j in(r+1):k
            g_j = g.coef[j]
            L_j = L.coef[k - j + 1]
            suma += (k - j)*g_j*L_j
        end
        L.coef[k - r + 1] = (k*g_k - suma)/(g_r*(k - r))
    end
    return L
end

log (generic function with 20 methods)

Para las funciones $Cos(x)$ y $Sin(x)$ recordamos la fórmula de Euler:
$$
e^{i\theta} = cos \theta + i sin \theta
$$
Así:
$$Cos(g(x)) = Re \{ e^{ig(x)} \} $$
$$Sin(g(x)) = Im \{ e^{ig(x)} \} $$

Anteriormente hemos obtenido la expresión de los coeficientes de la expresión de Taylor para la exponencial de una función. 
Con esto, nos aventuramos a proponer que:
$$\{ Cos (g(x)) \}_{k}  = C_k = Re\{E_k\} $$
$$\{ Sin (g(x)) \}_{k}  = S_k = Im\{E_k\} $$

In [17]:
#Implementamos la función exponencial

import Base.exp
function exp(a::Taylor)
    E = Taylor(zeros(typeof(exp(a.coef[1])),a.order))     #Esta exponencial permite coeficientes complejos
    r = 0
    for i in 1:a.order #Verificamos que el Taylor proporcionado sea distinto de cero
        if a.coef[i] != 0
            r = i
            break
        end
    end
    @assert r != 0 "Necesitamos un polinomio distinto de cero"
    E.coef[1] = exp(a.coef[1])
    for k in 2:a.order
        suma = 0
        for j in 1:k-1
            suma += E.coef[j] * (k - j) * a.coef[k-j+1]
        end
        E.coef[k] = suma / (k-1)
    end
    E
end

exp (generic function with 15 methods)

In [18]:
# Sera util enseñarle a computadora a separar las componentes real y complejas de los coeficientes de un taylor.
import Base: real, imag

In [19]:
function real(f::Taylor)
    h = Taylor(zeros(typeof(real(f.coef[1])),f.order))
    
    for k in 1:f.order
        h.coef[k] = real(f.coef[k])
    end
    return h
end

function imag(f::Taylor)
    h = Taylor(zeros(typeof(real(f.coef[1])),f.order))
    
    for k in 1:f.order
        h.coef[k] = imag(f.coef[k])
    end
    return h
end

imag (generic function with 17 methods)

In [20]:
# Ponemos el Seno y el Coseno como nosotros entendimos

import Base: sin, cos

function cos(f::Taylor)
    t1 = im*f
    ex = exp(t1)
    return real(ex)
end

function sin(f::Taylor)
    t1 = im*f
    ex = exp(t1)
    return imag(ex)
end


sin (generic function with 14 methods)

Y los correspondientes Tests:

In [21]:
g = Taylor([e, e, e])
t = Taylor([1, 1, 0]) #Equivalente al polinomio 1 + x 
t1 = Taylor([pi]) #Taylor de orden cero con término independiente pi
t2 = Taylor([pi/2.0])

@test log(g) == Taylor([1.0, 2.0, -1.0])
@test t^2 == Taylor([1.0,2.0,1.0]) #t^2 = 1 + 2x + x^2
@test cos(t1) == Taylor([-1.0])  #cos(pi) = -1
@test sin(t2) == Taylor([1.0]) #sin(pi/2) = 1

Todo lo pusimos en los archivos __Taylor.jl__ y __runtest_taylor.jl__ .